In [50]:
import colorama
from colorama import Fore

import random
import string

VOWELS = 'aeiou'
CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
HAND_SIZE = 7

SCRABBLE_LETTER_VALUES = {
    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, 'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1, 'o': 1, 'p': 3, 'q': 10, 'r': 1, 's': 1, 't': 1, 'u': 1, 'v': 4, 'w': 4, 'x': 8, 'y': 4, 'z': 10
}


WORDLIST_FILENAME = "words.txt"

In [51]:
def loadWords():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    """
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # wordList: list of strings
    wordList = []
    for line in inFile:
        wordList.append(line.strip().lower())
    print("  ", len(wordList), "words loaded.")
    return wordList

In [52]:
def getFrequencyDict(sequence):
    """
    Returns a dictionary where the keys are elements of the sequence
    and the values are integer counts, for the number of times that
    an element is repeated in the sequence.

    sequence: string or list
    return: dictionary
    """
    # freqs: dictionary (element_type -> int)
    freq = {}
    for x in sequence:
        freq[x] = freq.get(x,0) + 1
    return freq

### Scoring a word

In [53]:
def getWordScore(word, n):
    """
    Returns the score for a word. Assumes the word is a valid word.

    The score for a word is the sum of the points for letters in the
    word, multiplied by the length of the word, PLUS 50 points if all n
    letters are used on the first turn.

    Letters are scored as in Scrabble; A is worth 1, B is worth 3, C is
    worth 3, D is worth 2, E is worth 1, and so on (see SCRABBLE_LETTER_VALUES)

    word: string (lowercase letters)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    returns: int >= 0
    """
    #keep track of total score
    total = 0
    
    #get score for each letter in user's word
    for letter in word:
        for key in SCRABBLE_LETTER_VALUES:
            #find score for letter in dict
            if letter == key:
                total += SCRABBLE_LETTER_VALUES.get(key)
    
    #multiply by word length, as given by rules
    total = total * len(word)
    
    #if all letters used, 50 bonus points
    if len(word) == n:
        total += 50
        
    #total score
    return total

### Displaying a Hand

In [54]:
def displayHand(hand):
    """
    Displays the letters currently in the hand.

    For example:
    >>> displayHand({'a':1, 'x':2, 'l':3, 'e':1})
    Should print out something like:
       a x x l l l e
    The order of the letters is unimportant.

    hand: dictionary (string -> int)
    """
    
    #return every letter in hand
    for letter in hand.keys():
        for j in range(hand[letter]):
             print(Fore.MAGENTA + letter,end=" ")      # print all on the same line
    print()                             # print an empty line

### Deal hand

In [55]:
def dealHand(n):
    """
    Returns a random hand containing n lowercase letters.
    At least n/3 the letters in the hand should be VOWELS.

    Hands are represented as dictionaries. The keys are
    letters and the values are the number of times the
    particular letter is repeated in that hand.

    n: int >= 0
    returns: dictionary (string -> int)
    """
    #hand - dictionary
    hand={}
    
    #minimum amount of vowels are used
    numVowels = n // 3
    
    #get random vowels
    for i in range(numVowels):
        x = VOWELS[random.randrange(0,len(VOWELS))]
        hand[x] = hand.get(x, 0) + 1
        
    #get random consonants
    for i in range(numVowels, n):    
        x = CONSONANTS[random.randrange(0,len(CONSONANTS))]
        hand[x] = hand.get(x, 0) + 1
        
    #return hand of n letters
    return hand

### Update a hand by removing letters

In [56]:
def updateHand(hand, word):
    """
    Assumes that 'hand' has all the letters in word.
    In other words, this assumes that however many times
    a letter appears in 'word', 'hand' has at least as
    many of that letter in it. 

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.

    word: string
    hand: dictionary (string -> int)    
    returns: dictionary (string -> int)
    """
    
    #copy of hand dictionary that can be updated
    newHand = hand.copy()
    
    #if letter is used in word, remove from hand
    for letter in word:
        newHand[letter] -= 1
        if newHand[letter] == 0:
            del(newHand[letter])
    
    #updated newHand, does not change hand
    return newHand

### Test a word's validity

In [57]:
def isValidWord(word, hand, wordList):
    """
    Returns True if word is in the wordList and is entirely
    composed of letters in the hand. Otherwise, returns False.

    Does not mutate hand or wordList.
   
    word: string
    hand: dictionary (string -> int)
    wordList: list of lowercase strings
    """
    
    #check if word in wordList
    if word not in wordList:
        return False
    
    #get number of letters in user's word
    freq = getFrequencyDict(word)
    
    #check if letters used are in hand
    for letter in freq:
        if hand.get(letter,0) < freq[letter]:
            return False
    return True

### Calculate hand len

In [58]:
def calculateHandlen(hand):
    """ 
    Returns the length (number of letters) in the current hand.
    
    hand: dictionary (string-> int)
    returns: integer
    """
    
    #keep track of hand length
    handLen = 0
    
    for key in hand:
        handLen += hand.get(key)
    return handLen

### Play Hand

In [59]:
def playHand(hand, wordList, n):
    """
    Allows the user to play the given hand, as follows:

    * The hand is displayed.
    * The user may input a word or a single period (the string ".") 
      to indicate they're done playing
    * Invalid words are rejected, and a message is displayed asking
      the user to choose another word until they enter a valid word or "."
    * When a valid word is entered, it uses up letters from the hand.
    * After every valid word: the score for that word is displayed,
      the remaining letters in the hand are displayed, and the user
      is asked to input another word.
    * The sum of the word scores is displayed when the hand finishes.
    * The hand finishes when there are no more unused letters or the user
      inputs a "."

      hand: dictionary (string -> int)
      wordList: list of lowercase strings
      n: integer (HAND_SIZE; i.e., hand size required for additional points)
      
    """

    # Keep track of the total score
    total = 0
    
    # As long as there are still letters left in the hand:
    while len(hand) > 0:
        # Display the hand
        print(Fore.BLUE + "Current Hand: ")
        print("\n")
        displayHand(hand)
        # Ask user for input
        userInput = input(Fore.BLUE + "Enter word or single period to end game: ") 
        # If the input is a single period:
        if userInput == ".":
            break
            # End the game (break out of the loop)

        else:
        # Otherwise (the input is not a single period):
        
            # If the word is not valid:
            if isValidWord(userInput,hand,wordList) == False:
                print(Fore.BLUE + "Word is invalid \n")
                print("\n")
                # Reject invalid word (print a message followed by a blank line)

            # Otherwise (the word is valid):
            else:
                total += getWordScore(userInput,n)
                print(Fore.BLUE + "Points Earned:",getWordScore(userInput,n),Fore.BLUE + "Total Points:",total)
                print("\n")
                # Tell the user how many points the word earned, and the updated total score, in one line followed by a blank line
                
                # Update the hand 
                hand = updateHand(hand,userInput)
    
    # Game is over (user entered a '.' or ran out of letters), so tell user the total score
    if len(hand) == 0:
        print(Fore.GREEN + "Out of letters! Total score:",total)
    else:
        print(Fore.GREEN + "Game Over! Total Score:",total)

### Playing a game

In [60]:
def playGame(wordList):
    hand = []
    
    #want the game to continue until "exit"
    while True:
        user_choice = input(Fore.BLUE + 'Enter n to deal a new hand, r to replay the last hand, or e to end game: ')
        if user_choice == 'e':
            print(Fore.BLUE + "Play again soon!")
            break
        else:
            if user_choice == 'n':
                hand = dealHand(HAND_SIZE)
                playHand(hand, wordList, HAND_SIZE)
                
            #reuse last hand
            elif user_choice == 'r' and len(hand) != 0:
                playHand(hand, wordList, HAND_SIZE)
            elif user_choice == 'r':
                print(Fore.RED + 'You have not played a hand yet. Please play a new hand first!')
                print("\n")
            else:
                print(Fore.RED + 'Invalid command.')
                print("\n")

In [62]:
wordList = loadWords()
playGame(wordList)

Loading word list from file...
   83667 words loaded.
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
Current Hand: 


u o j n c p r 
Enter word or single period to end game: not
Word is invalid 



Current Hand: 


u o j n c p r 
Enter word or single period to end game: .
Game Over! Total Score: 0
Enter n to deal a new hand, r to replay the last hand, or e to end game: r
Current Hand: 


u o j n c p r 
Enter word or single period to end game: ron
Word is invalid 



Current Hand: 


u o j n c p r 
Enter word or single period to end game: .
Game Over! Total Score: 0
Enter n to deal a new hand, r to replay the last hand, or e to end game: e
Play again soon!
